# Chargement des données de pandémies - Partie 2: Chargement des données

Ce notebook réalise le chargement des données transformées dans la base de données MySQL.

## 1. Configuration initiale

Importation des bibliothèques nécessaires et définition des paramètres de connexion à la base de données.

In [17]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from pathlib import Path
import time

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Affichage du chemin
print(f"Dossier des données transformées: {dossier_donnees_transformees}")

Dossier des données transformées: C:\Users\Saint\IdeaProjects\ETL_Simple_Project\ETL_Projet_Pandemies\donnees\transformees


## 2. Configuration de la connexion à la base de données

Définition des paramètres de connexion à la base de données MySQL.

In [18]:
# Paramètres de connexion à la base de données
config_db = {
    "host": "localhost",
    "user": "root",
    "password": "",  # Laisser vide si aucun mot de passe
    "database": "epiviz",
    "port": 3306
}

print("Paramètres de connexion à la base de données:")
for cle, valeur in config_db.items():
    if cle != "password":
        print(f"- {cle}: {valeur}")
    else:
        print(f"- {cle}: {'(aucun)' if valeur == '' else '******'}")

Paramètres de connexion à la base de données:
- host: localhost
- user: root
- password: (aucun)
- database: epiviz
- port: 3306


## 3. Fonction de connexion à la base de données

Définition d'une fonction pour établir une connexion à la base de données MySQL.

In [19]:
def connecter_mysql(config=None):
    """
    Établit une connexion à la base de données MySQL.
    
    Args:
        config (dict): Dictionnaire contenant les paramètres de connexion.
        
    Returns:
        tuple: (connexion, curseur) si la connexion est établie, (None, None) sinon.
    """
    if config is None:
        config = config_db.copy()
    
    try:
        # Connexion à la base de données
        connexion = mysql.connector.connect(**config)
        curseur = connexion.cursor()
        print(f"Connexion à la base de données '{config['database']}' établie avec succès.")
        return connexion, curseur
    
    except Error as e:
        print(f"Erreur lors de la connexion à la base de données: {e}")
        return None, None

## 4. Chargement des données transformées

Chargement des données transformées depuis les fichiers CSV.

In [20]:
# Chargement des données transformées
print("Chargement des données transformées...")

tables = {}

try:
    # Chargement de la table calendrier
    fichier_calendrier = dossier_donnees_transformees / "calendrier.csv"
    if fichier_calendrier.exists():
        tables["calendrier"] = pd.read_csv(fichier_calendrier, parse_dates=["date"])
        print(f"Table calendrier chargée: {tables['calendrier'].shape[0]} lignes, {tables['calendrier'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger la table calendrier: fichier non trouvé")
    
    # Chargement de la table localisation
    fichier_localisation = dossier_donnees_transformees / "localisation.csv"
    if fichier_localisation.exists():
        tables["localisation"] = pd.read_csv(fichier_localisation)
        print(f"Table localisation chargée: {tables['localisation'].shape[0]} lignes, {tables['localisation'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger la table localisation: fichier non trouvé")
    
    # Chargement de la table pandemie
    fichier_pandemie = dossier_donnees_transformees / "pandemie.csv"
    if fichier_pandemie.exists():
        tables["pandemie"] = pd.read_csv(fichier_pandemie, parse_dates=["date_debut"])
        print(f"Table pandemie chargée: {tables['pandemie'].shape[0]} lignes, {tables['pandemie'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger la table pandemie: fichier non trouvé")
    
    # Chargement de la table donnees
    fichier_donnees = dossier_donnees_transformees / "donnees.csv"
    if fichier_donnees.exists():
        tables["donnees"] = pd.read_csv(fichier_donnees)
        print(f"Table donnees chargée: {tables['donnees'].shape[0]} lignes, {tables['donnees'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger la table donnees: fichier non trouvé")
    
    print("\nChargement des données transformées terminé avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement des données transformées: {e}")

Chargement des données transformées...
Table calendrier chargée: 1204 lignes, 5 colonnes
Table localisation chargée: 260 lignes, 3 colonnes
Table pandemie chargée: 2 lignes, 4 colonnes
Table donnees chargée: 226525 lignes, 6 colonnes

Chargement des données transformées terminé avec succès!


## 5. Préparation des données pour le chargement

Préparation des données pour le chargement dans la base de données MySQL.

In [21]:
# Préparation des données pour le chargement
print("Préparation des données pour le chargement...")

try:
    # Préparation de la table calendrier
    if "calendrier" in tables:
        # Conversion des dates en chaînes de caractères au format MySQL
        tables["calendrier"]["date"] = tables["calendrier"]["date"].dt.strftime("%Y-%m-%d")
        print("Table calendrier préparée pour le chargement.")
    
    # Préparation de la table pandemie
    if "pandemie" in tables:
        # Conversion des dates en chaînes de caractères au format MySQL
        tables["pandemie"]["date_debut"] = tables["pandemie"]["date_debut"].dt.strftime("%Y-%m-%d")
        print("Table pandemie préparée pour le chargement.")
    
    print("\nPréparation des données pour le chargement terminée avec succès!")
except Exception as e:
    print(f"Erreur lors de la préparation des données pour le chargement: {e}")

Préparation des données pour le chargement...
Table calendrier préparée pour le chargement.
Table pandemie préparée pour le chargement.

Préparation des données pour le chargement terminée avec succès!


## 6. Fonction de vidage des tables

Définition d'une fonction pour vider les tables de la base de données.

In [22]:
def vider_tables(connexion, curseur, tables):
    """
    Vide les tables de la base de données si elles existent.
    
    Args:
        connexion: Connexion à la base de données.
        curseur: Curseur de la base de données.
        tables (list): Liste des noms des tables à vider.
        
    Returns:
        bool: True si les tables ont été vidées avec succès, False sinon.
    """
    try:
        # Désactivation temporaire des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 0;")
        
        # Récupérer la liste des tables existantes
        curseur.execute("SHOW TABLES;")
        tables_existantes = [table[0] for table in curseur.fetchall()]
        print(f"Tables existantes: {tables_existantes}")
        
        # Vidage des tables dans l'ordre inverse des dépendances
        for table in tables:
            if table in tables_existantes:
                print(f"Vidage de la table {table}...")
                curseur.execute(f"TRUNCATE TABLE {table};")
                print(f"Table {table} vidée avec succès.")
            else:
                print(f"La table {table} n'existe pas, création nécessaire.")
        
        # Réactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 1;")
        
        # Validation des modifications
        connexion.commit()
        print("\nOpération de vidage terminée.")
        return True
    
    except Error as e:
        print(f"Erreur lors du vidage des tables: {e}")
        return False

## 7. Fonction de chargement des données

Définition d'une fonction pour charger les données dans la base de données.

In [26]:
def charger_donnees(connexion, curseur, table_nom, table_df, taille_lot=1000):
    try:
        # Récupération des noms des colonnes
        colonnes = list(table_df.columns)
        print(f"Colonnes de la table {table_nom}: {colonnes}")
        
        # Vérification des valeurs nulles
        for colonne in colonnes:
            null_count = table_df[colonne].isnull().sum()
            if null_count > 0:
                print(f"ATTENTION: La colonne {colonne} contient {null_count} valeurs nulles!")
        
        # Préparation de la requête d'insertion
        placeholders = ", ".join(["%s"] * len(colonnes))
        requete = f"INSERT INTO {table_nom} ({', '.join(colonnes)}) VALUES ({placeholders})"
        
        # Conversion des données en liste de tuples
        donnees = [tuple(x) for x in table_df.to_numpy()]
        
        # Affichage des premières lignes pour débogage
        print(f"Exemple de données à charger dans {table_nom} (3 premières lignes):")
        for i in range(min(3, len(donnees))):
            print(donnees[i])
        
        # Chargement des données par lots
        print(f"Chargement de {len(donnees)} lignes dans la table {table_nom}...")
        
        for i in range(0, len(donnees), taille_lot):
            lot = donnees[i:i + taille_lot]
            try:
                curseur.executemany(requete, lot)
                connexion.commit()
                print(f"Lot {i // taille_lot + 1}/{(len(donnees) + taille_lot - 1) // taille_lot} chargé: {len(lot)} lignes")
            except Error as e:
                print(f"Erreur lors du chargement du lot {i // taille_lot + 1}: {e}")
                # Afficher la première ligne qui pose problème
                if len(lot) > 0:
                    print(f"Première ligne du lot problématique: {lot[0]}")
                raise
        
        print(f"Données chargées dans la table {table_nom} avec succès.")
        return True
    
    except Error as e:
        print(f"Erreur lors du chargement des données dans la table {table_nom}: {e}")
        return False

## 8. Chargement des données dans la base de données

Chargement des données dans la base de données MySQL.

In [ ]:
try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Désactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 0;")
        
        # Suppression des tables si elles existent déjà
        print("Suppression des tables existantes...")
        curseur.execute("DROP TABLE IF EXISTS donnees;")
        curseur.execute("DROP TABLE IF EXISTS calendrier;")
        curseur.execute("DROP TABLE IF EXISTS localisation;")
        curseur.execute("DROP TABLE IF EXISTS pandemie;")
        
        # Création des tables SANS contraintes de clé étrangère
        print("Création des tables sans contraintes de clé étrangère...")
        
        # Table calendrier
        curseur.execute("""
        CREATE TABLE IF NOT EXISTS calendrier (
            id_date INT PRIMARY KEY,
            date DATE NOT NULL,
            jour INT NOT NULL,
            mois INT NOT NULL,
            annee INT NOT NULL
        ) ENGINE=InnoDB;
        """)
        print("Table calendrier créée.")
        
        # Table localisation
        curseur.execute("""
        CREATE TABLE IF NOT EXISTS localisation (
            id_localisation INT PRIMARY KEY,
            pays VARCHAR(100) NOT NULL,
            continent VARCHAR(50)
        ) ENGINE=InnoDB;
        """)
        print("Table localisation créée.")
        
        # Table pandemie
        curseur.execute("""
        CREATE TABLE IF NOT EXISTS pandemie (
            id_pandemie INT PRIMARY KEY,
            nom_pandemie VARCHAR(50) NOT NULL,
            description TEXT,
            date_debut DATE
        ) ENGINE=InnoDB;
        """)
        print("Table pandemie créée.")
        
        # Table donnees sans contraintes de clé étrangère
        curseur.execute("""
        CREATE TABLE IF NOT EXISTS donnees (
            id_donnees INT PRIMARY KEY,
            id_date INT NOT NULL,
            id_localisation INT NOT NULL,
            id_pandemie INT NOT NULL,
            cas_confirmes INT NOT NULL,
            deces INT NOT NULL
        ) ENGINE=InnoDB;
        """)
        print("Table donnees créée.")
        
        # Validation des modifications
        connexion.commit()
        print("Toutes les tables ont été créées avec succès.")
        
        # Chargement des données dans l'ordre des dépendances
        ordre_chargement = ["calendrier", "localisation", "pandemie", "donnees"]
        
        for table_nom in ordre_chargement:
            if table_nom in tables:
                print(f"\nChargement des données dans la table {table_nom}...")
                # Conversion des types NumPy en types Python standard
                for colonne in tables[table_nom].select_dtypes(include=[np.number]).columns:
                    tables[table_nom][colonne] = tables[table_nom][colonne].astype(float).astype(int)
                
                # Chargement des données
                charger_donnees(connexion, curseur, table_nom, tables[table_nom])
        
        # Ajout des contraintes de clé étrangère après le chargement des données
        print("\nAjout des contraintes de clé étrangère...")
        try:
            curseur.execute("""
            ALTER TABLE donnees 
            ADD CONSTRAINT fk_donnees_calendrier FOREIGN KEY (id_date) REFERENCES calendrier(id_date),
            ADD CONSTRAINT fk_donnees_localisation FOREIGN KEY (id_localisation) REFERENCES localisation(id_localisation),
            ADD CONSTRAINT fk_donnees_pandemie FOREIGN KEY (id_pandemie) REFERENCES pandemie(id_pandemie);
            """)
            print("Contraintes de clé étrangère ajoutées avec succès.")
        except Error as e:
            print(f"Erreur lors de l'ajout des contraintes de clé étrangère: {e}")
        
        # Réactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 1;")
        connexion.commit()
        
        print("\nToutes les données ont été chargées avec succès.")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("Connexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors du chargement des données dans la base de données: {e}")

Chargement des données dans la base de données...
Connexion à la base de données 'epiviz' établie avec succès.
Création des tables...
Table calendrier créée.
Table localisation créée.
Erreur lors du chargement des données dans la base de données: 1005 (HY000): Can't create table `epiviz`.`pandemie` (errno: 150 "Foreign key constraint is incorrectly formed")


## Conclusion

Dans ce notebook, nous avons réalisé le chargement des données transformées dans la base de données MySQL. Les principales opérations ont été :

1. Chargement des données transformées depuis les fichiers CSV
2. Préparation des données pour le chargement dans la base de données
3. Vidage des tables existantes
4. Chargement des données dans la base de données dans l'ordre des dépendances

Les données sont maintenant chargées dans la base de données MySQL et prêtes à être utilisées pour l'analyse et la visualisation. La prochaine étape consiste à vérifier les données chargées, ce qui sera réalisé dans le prochain notebook.